In [1]:
import csv
import pickle
import random
import re
import pandas as pd

# 1 data augmentation (using koeda)

KoEDA  
- `영어용으로 구현된 Easy data augmentation 과 An Easier Data Augmentation 프로젝트를 한글용으로 재구성한 프로젝트입니다.`

🎯 EasyDataAugmentation 파라미터
 - `p = (alpha_sr,alpha_ri,alpha_rs,prob_rd)` 은 <br> SR, RI , RS , RD 에 대한 각각의 변환을 어느정도 비율로 할 것인지 결정
 > - Synonym Replacement __(SR)__ : 유의어 교체
 > - Random Insertion __(RI)__ : 임의 단어 삽입
 > - Random Swap __(RS)__ : 두 단어 위치 변경
 > - Random Deletion __(RD)__ : 임의 단어 삭제
 - morpheme_analyzer 는 사용될 형태소 분석기를 지정하는 파라미터로, <br> ["Okt", "Kkma", "Komoran", "Mecab", "Hannanum"] 중 하나를 선택할 수 있다.  <br>__(단, 일부는 설치 필요하며, 각각 형태소를 나누는 기준이 다르다. )__

In [1]:
from koeda import EDA
from koeda import EasyDataAugmentation


eda = EasyDataAugmentation(
    morpheme_analyzer="Okt"
    )

eda1 = EasyDataAugmentation(
    morpheme_analyzer="Kkma"
    )

eda2 = EasyDataAugmentation(
    morpheme_analyzer="Komoran"
    )

eda3 = EasyDataAugmentation(
    morpheme_analyzer="Mecab"
    )

eda4 = EasyDataAugmentation(
    morpheme_analyzer="Hannanum"
    )

text = "아버지가 방에 들어가신다"

result = eda(text)
print(result)
# 아버지가 정실에 들어가신다

result = eda(text,p=(0.9, 0.5, 0.5, 0.0), repetition=3)
print(result)
# ['아버지가 객실 아빠 안방 방에 정실 들어가신다', '아버지가 탈의실 방 휴게실 에 안방 탈의실 들어가신다']

result1 = eda1(text)
print(result1)
# 아버지가 정실에 들어가신다

result1 = eda1(text,p=(0.5, 0.5, 0.5, 0.0), repetition=3)
print(result1)
# ['아버지가 객실 아빠 안방 방에 정실 들어가신다', '아버지가 탈의실 방 휴게실 에 안방 탈의실 들어가신다']

result2 = eda2(text)
print(result2)
# 아버지가 정실에 들어가신다

result2 = eda2(text,p=(0.5, 0.5, 0.5, 0.0), repetition=3)
print(result2)
# ['아버지가 객실 아빠 안방 방에 정실 들어가신다', '아버지가 탈의실 방 휴게실 에 안방 탈의실 들어가신다']

result3 = eda3(text)
print(result3)
# 아버지가 정실에 들어가신다

result3 = eda3(text,p=(0.9, 0.9, 0.0, 0.0), repetition=2)
print(result)
# ['아버지가 객실 아빠 안방 방에 정실 들어가신다', '아버지가 탈의실 방 휴게실 에 안방 탈의실 들어가신다']

result4 = eda4(text)
print(result4)
# 아버지가 정실에 들어가신다

result4 = eda4(text,p=(0.5, 0.5, 0.5, 0.0), repetition=3)
print(result4)
# ['아버지가 객실 아빠 안방 방에 정실 들어가신다', '아버지가 탈의실 방 휴게실 에 안방 탈의실 들어가신다']

아버지가 휴게실에 들어가신다
['아버지방 에가 들어가신다', '아버지가 방에 안방 휴게실 들어가신다', '아버지가 침실에 들어가신다']
탈의실 아버지가 방에 들어가시ㄴ다
['아버지가 골방에 들어가시ㄴ다', '아버지가 방에 들어가시ㄴ다', '침투하 아버지가 방 침실 골방 에 들어가시ㄴ다']
아버지가 방에 들어가시ㄴ다
['부친가 방에 들어가시ㄴ다', 'ㄴ다가 방시 에들어가아버지', '아버지가 시에방 들어가ㄴ다']
들어가가 방에 아버지신다
['아버지방 에가 들어가신다', '아버지가 방에 안방 휴게실 들어가신다', '아버지가 침실에 들어가신다']
아버지가 방에 들어가시ㄴ다이
['침실 평원 독방 부모 아버지가 방에 들어가이시ㄴ다', '아버지가 방에 들어가이시ㄴ다', '아버지가 방에 들어가이시ㄴ다']


In [1]:
from koeda import EasyDataAugmentation

def augment_text_data_with_EDA(text,repetition):
    """입력된 문장에 대해서 EDA를 통해 데이터 증강"""
    eda = EasyDataAugmentation(
        morpheme_analyzer="Okt"
        )

    result = eda(text,p=(0.5, 0.5, 0.5, 0.5), repetition=repetition)

    # 증강 결과 출력
    print("원문: " , text)
    print("--"*100)
    for i in range(repetition):
        print(f"증강문{i+1}: ", result[i])
    # return result

# 2 data augmentation (using SR / KWN wordnet)

In [2]:
# WordNet 로드
with open("wordnet.pickle", "rb") as f:
    wordnet = pickle.load(f)

def replace_entities_with_placeholders(text, placeholder_prefix="#REPL#"):
    entities = re.findall(r"#.+?#", text)
    for i, entity in enumerate(entities):
        text = text.replace(entity, f"{placeholder_prefix}{i}#", 1)
    return text, entities

def replace_placeholders_with_entities(text, entities, placeholder_prefix="#REPL#"):
    for i, entity in enumerate(entities):
        text = text.replace(f"{placeholder_prefix}{i}#", entity, 1)
    return text

def augment_text(text, wordnet):
    text, entities = replace_entities_with_placeholders(text)
    
    words = text.split()
    for i, word in enumerate(words):
        if word in wordnet:
            synonyms = wordnet[word]
            if synonyms:
                words[i] = random.choice(synonyms)
                break
    
    augmented_text = " ".join(words)
    augmented_text = replace_placeholders_with_entities(augmented_text, entities)
    
    return augmented_text

input_file = "../data/augment/last_back_train.csv"
output_file = "../data/augment/back_kwn37500_train.csv"
augmented_data = []

with open(input_file, encoding='utf-8') as csvfile:
    reader = list(csv.DictReader(csvfile))  # 데이터를 메모리에 로드하기 위해 list로 변환
    original_data_len = len(reader)
    target_augment_count = 37500 - original_data_len  # 증강을 원하는 목표 수
    
    # 기존 데이터를 augmented_data에 추가
    for row in reader:
        augmented_data.append(row)
    
    # 데이터 증강 시작
    while len(augmented_data) < 37500:
        for row in reader:
            # 대화와 요약 증강
            augmented_dialogue = augment_text(row['dialogue'], wordnet)
            # augmented_summary = augment_text(row['summary'], wordnet)
            augmented_row = {
                'fname': row['fname'],
                'dialogue': augmented_dialogue,
                'summary': row['summary'],
                'topic': row['topic']
            }
            
            # 증강된 데이터 추가
            augmented_data.append(augmented_row)
            
            # 목표 데이터 수에 도달했는지 확인
            if len(augmented_data) >= 37500:
                break

# 증강된 데이터를 CSV 파일로 저장
with open(output_file, mode='w', encoding='utf-8', newline='') as f:
    fieldnames = ['fname', 'dialogue', 'summary', 'topic']
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for row in augmented_data[:37500]:  # 최대 50,000개 데이터만 저장
        writer.writerow(row)


In [3]:
train_new = pd.read_csv("../data/augment/kwn70000_train.csv")
train_new

,fname,dialogue,summary,topic
0,train_0,"#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나...","스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니...",건강검진 받기
1,train_1,"#Person1#: 안녕하세요, 파커 부인, 어떻게 지내셨나요?\n#Person2#...",파커 부인이 리키를 데리고 백신 접종을 하러 갔다. 피터스 박사는 기록을 확인한 후...,백신
2,train_2,"#Person1#: 실례합니다, 열쇠 한 묶음 보셨나요?\n#Person2#: 어떤...","#Person1#은 열쇠 한 묶음을 찾고 있고, 그것을 찾기 위해 #Person2#...",열쇠 찾기
3,train_3,#Person1#: 왜 너는 여자친구가 있다는 걸 말해주지 않았어?\n#Person...,#Person1#은 #Person2#가 여자친구가 있고 그녀와 결혼할 것이라는 사실...,여자친구가 있다
4,train_4,"#Person1#: 안녕, 숙녀분들! 오늘 밤 당신들은 정말 멋져 보여. 이 춤을 ...",말릭이 니키에게 춤을 요청한다. 말릭이 발을 밟는 것을 신경 쓰지 않는다면 니키는 ...,댄스
...,...,...,...,...
69995,train_7710,"#Person1#: 이제, 선씨, 사고를 명확하게 보셨나요? #Person2#: 아...",선씨는 사고를 목격하고 #Person1#에게 자세한 정보를 알려줍니다.,사고 목격자
69996,train_7711,"#Person1#: 전산 사용법을 배우기 시작해봅시다, 퍼블러. 컴퓨터에는 하드웨어...","#Person1#과 퍼블러는 컴퓨터 사용법을 공부하고 있으며, #Person1#은 ...",컴퓨터
69997,train_7712,"#Person1#: 좋은 아침입니다, 도와드릴 일이 있나요? #Person2#: 아...","구찌와 메이가 옷을 사러 쇼핑을 갔습니다. 처음에는 메이가 무심하게 행동하지만, 구...",옷 구매
69998,train_7713,#Person1#: 친구의 결혼식을 폐해 어떤 천을 골라 슈트를 만들어 주실 수 있...,#Person1#은 친구의 결혼식을 위해 갈색 줄무늬 슈트를 만들고 싶어합니다. #...,옷 선택


# 3 Only kwn augmentation

In [2]:
import csv
import pickle
import re
import random

# WordNet 로드
with open("wordnet.pickle", "rb") as f:
    wordnet = pickle.load(f)

# 기존 함수들...
def replace_entities_with_placeholders(text, placeholder_prefix="#REPL#"):
    entities = re.findall(r"#.+?#", text)
    for i, entity in enumerate(entities):
        text = text.replace(entity, f"{placeholder_prefix}{i}#", 1)
    return text, entities

def replace_placeholders_with_entities(text, entities, placeholder_prefix="#REPL#"):
    for i, entity in enumerate(entities):
        text = text.replace(f"{placeholder_prefix}{i}#", entity, 1)
    return text

def augment_text(text, wordnet):
    text, entities = replace_entities_with_placeholders(text)
    
    words = text.split()
    for i, word in enumerate(words):
        if word in wordnet:
            synonyms = wordnet[word]
            if synonyms:
                words[i] = random.choice(synonyms)
                break
    
    augmented_text = " ".join(words)
    augmented_text = replace_placeholders_with_entities(augmented_text, entities)
    
    return augmented_text

input_file = "../data/augment/back_result_train.csv"
output_file = "../data/augment/back_kwn10000_augmented.csv"  # 변경된 파일명
new_augmented_data = []  # 새롭게 생성된 데이터만을 저장할 리스트

with open(input_file, encoding='utf-8') as csvfile:
    reader = list(csv.DictReader(csvfile))
    original_data_len = len(reader)
    target_augment_count = 10000  # 변경: 새롭게 생성할 데이터의 목표 수
    
    # 데이터 증강 시작
    while len(new_augmented_data) < target_augment_count:  # 변경: 새로운 리스트의 크기를 확인
        for row in reader:
            # 대화와 요약 증강 (요약은 증강하지 않기로 함)
            augmented_dialogue = augment_text(row['dialogue'], wordnet)
            augmented_row = {
                'fname': row['fname'],
                'dialogue': augmented_dialogue,
                'summary': row['summary'],  # 원본 요약 사용
                'topic': row['topic']
            }
            
            # 증강된 데이터 추가
            new_augmented_data.append(augmented_row)
            
            # 목표 데이터 수에 도달했는지 확인
            if len(new_augmented_data) >= target_augment_count:
                break

# 증강된 데이터를 CSV 파일로 저장
with open(output_file, mode='w', encoding='utf-8', newline='') as f:
    fieldnames = ['fname', 'dialogue', 'summary', 'topic']
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for row in new_augmented_data:  # 새로운 리스트의 데이터만 저장
        writer.writerow(row)


: 